# Computation of the RER CF for land use

Based on country-specific CFs, we compute the one of RER which is an average weighted by the surface area of each country (i.e., similarly as the global average was computed in the original method).

In [1]:
import pandas as pd
import country_converter as coco

In [2]:
country_surface = pd.read_csv('data/world_bank_country_surface_area.csv')
ef_cfs = pd.read_excel('data/EF-LCIAMethod_CF(EF-v3.1).xlsx', sheet_name='lciamethods_CF', keep_default_na=False, na_values='')

In [3]:
country_surface['Country code (ISO2)'] = coco.convert(names=list(country_surface['Country Name']), to='ISO2')

Africa Eastern and Southern not found in regex
Africa Western and Central not found in regex
Arab World not found in regex
Central Europe and the Baltics not found in regex
Channel Islands not found in regex
Caribbean small states not found in regex
East Asia & Pacific ( not found in regex
Early-demographic dividend not found in regex
East Asia & Pacific not found in regex
Europe & Central Asia ( not found in regex
Europe & Central Asia not found in regex
Euro area not found in regex
European Union not found in regex
Fragile and conflict affected situations not found in regex
High income not found in regex
Heavily indebted poor countries (HIPC) not found in regex
IBRD only not found in regex
IDA & IBRD total not found in regex
IDA total not found in regex
IDA blend not found in regex
IDA only not found in regex
Not classified not found in regex
Latin America & Caribbean ( not found in regex
Latin America & Caribbean not found in regex
Least developed countries: UN classification not fo

In [4]:
ef_cfs_lu_glo = ef_cfs[(ef_cfs['LCIAMethod_name'] == 'Land use') & (ef_cfs.LCIAMethod_location.isna())]
ef_cfs_lu_lt = ef_cfs[(ef_cfs['LCIAMethod_name'] == 'Land use') & (ef_cfs['FLOW_class1'] == 'Land transformation')]
ef_cfs_lu_lo = ef_cfs[(ef_cfs['LCIAMethod_name'] == 'Land use') & (ef_cfs['FLOW_class1'] == 'Land occupation')]

In [5]:
european_countries_full_names = [
    'Liechtenstein',
    'Armenia',
    'Lithuania',
    'Austria',
    'Luxembourg',
    'Azerbaijan',
    'Malta',
    'Belarus',
    'Moldova',
    'Belgium',
    'Monaco',
    'Bosnia and Herzegovina',
    'Montenegro',
    'Bulgaria',
    'Netherlands',
    'Croatia',
    'Norway',
    'Cyprus',
    'Poland',
    'Czech Republic',
    'Portugal',
    'Denmark',
    'Romania',
    'Estonia',
    'Russia',
    'Finland',
    'San Marino',
    'Former Yugoslav Republic of Macedonia',
    'Serbia',
    'France',
    'Slovakia',
    'Georgia',
    'Slovenia',
    'Germany',
    'Spain',
    'Greece',
    'Sweden',
    'Hungary',
    'Sweden',
    'Iceland',
    'Switzerland',
    'Ireland',
    'Turkey',
    'Italy',
    'Ukraine',
    'Kosovo',
    'United Kingdom',
]

In [6]:
european_countries_iso2 = coco.convert(names=european_countries_full_names, to='ISO2')

In [7]:
ef_cfs_lu_lt_europe = ef_cfs_lu_lt[ef_cfs_lu_lt['LCIAMethod_location'].isin(european_countries_iso2)]
ef_cfs_lu_lo_europe = ef_cfs_lu_lo[ef_cfs_lu_lo['LCIAMethod_location'].isin(european_countries_iso2)]

In [8]:
ef_cfs_lu_lt_europe = ef_cfs_lu_lt_europe.merge(country_surface[['Country code (ISO2)', '2022']], how='left', left_on='LCIAMethod_location', right_on='Country code (ISO2)')
ef_cfs_lu_lo_europe = ef_cfs_lu_lo_europe.merge(country_surface[['Country code (ISO2)', '2022']], how='left', left_on='LCIAMethod_location', right_on='Country code (ISO2)')

In [9]:
# If a few locations have no surface area data, they are neglected
print(ef_cfs_lu_lt_europe[ef_cfs_lu_lt_europe['2022'].isna()].LCIAMethod_location.unique())
print(ef_cfs_lu_lo_europe[ef_cfs_lu_lo_europe['2022'].isna()].LCIAMethod_location.unique())
ef_cfs_lu_lt_europe.dropna(subset=['2022'], inplace=True)
ef_cfs_lu_lo_europe.dropna(subset=['2022'], inplace=True)

[]
[]


In [10]:
def weighted_average(df, value_column, weight_column):
    return (df[value_column] * df[weight_column]).sum() / df[weight_column].sum()

In [11]:
# Group by "FLOW_name" and apply the weighted average function
weighted_avg_lt_df = ef_cfs_lu_lt_europe.groupby('FLOW_name').apply(
    lambda x: weighted_average(x, 'CF EF3.1', '2022')
).reset_index(name='CF EF3.1')
weighted_avg_lo_df = ef_cfs_lu_lo_europe.groupby('FLOW_name').apply(
    lambda x: weighted_average(x, 'CF EF3.1', '2022')
).reset_index(name='CF EF3.1')

C:\Users\matth\AppData\Local\Temp\ipykernel_44488\96365720.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted_avg_lt_df = ef_cfs_lu_lt_europe.groupby('FLOW_name').apply(
C:\Users\matth\AppData\Local\Temp\ipykernel_44488\96365720.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted_avg_lo_df = ef_cfs_lu_lo_europe.groupby('FLOW_name').apply(


In [12]:
weighted_avg_lt_df['FLOW_class1'] = 'Land transformation'
weighted_avg_lo_df['FLOW_class1'] = 'Land occupation'

In [13]:
weighted_avg_df = pd.concat([weighted_avg_lt_df, weighted_avg_lo_df])

In [14]:
weighted_avg_df['FLOW_class0'] = 'Land use'
weighted_avg_df['LCIAMethod_location'] = 'RER'

In [15]:
weighted_avg_df = weighted_avg_df.merge(right=ef_cfs_lu_glo[['FLOW_uuid', 'FLOW_name', 'LCIAMethod_uuid EF3.1', 'LCIAMethod_name', 'FLOW_class0', 'FLOW_class1', 'FLOW_class2', 'LCIAMethod_derivation', 'LCIAMethod_direction']], on=['FLOW_name', 'FLOW_class0', 'FLOW_class1'], how='left')

In [16]:
weighted_avg_df

,FLOW_name,CF EF3.1,FLOW_class1,FLOW_class0,LCIAMethod_location,FLOW_uuid,LCIAMethod_uuid EF3.1,LCIAMethod_name,FLOW_class2,LCIAMethod_derivation,LCIAMethod_direction
0,from agriculture,-287.774023,Land transformation,Land use,RER,c0143eba-3daf-44ad-8d34-6e1d34128ace,b2ad6890-c78d-11e6-9d9d-cec0c932ce01,Land use,NaN,Calculated,Input
1,"from agriculture, mosaic",-287.774023,Land transformation,Land use,RER,aeea42cc-4215-465f-981f-ae2ab387fbb9,b2ad6890-c78d-11e6-9d9d-cec0c932ce01,Land use,NaN,Calculated,Input
2,from arable,-290.360454,Land transformation,Land use,RER,9699a63f-c966-43a4-816c-b1fbbf0a3e88,b2ad6890-c78d-11e6-9d9d-cec0c932ce01,Land use,NaN,Calculated,Input
3,"from arable, fallow",-292.947479,Land transformation,Land use,RER,67055318-bad9-420f-ae31-9ba54456518d,b2ad6890-c78d-11e6-9d9d-cec0c932ce01,Land use,NaN,Calculated,Input
4,"from arable, flooded crops",-280.032433,Land transformation,Land use,RER,d31e09db-9fc4-4842-8366-14dae068b402,b2ad6890-c78d-11e6-9d9d-cec0c932ce01,Land use,NaN,Calculated,Input
...,...,...,...,...,...,...,...,...,...,...,...
166,"urban, green areas",36.115544,Land occupation,Land use,RER,42378359-15eb-4e60-8f72-b796b9d71e69,b2ad6890-c78d-11e6-9d9d-cec0c932ce01,Land use,NaN,Calculated,Input
167,urban/industrial fallow,131.437295,Land occupation,Land use,RER,1c6a1585-0329-4134-aa24-b0aca0c5af11,b2ad6890-c78d-11e6-9d9d-cec0c932ce01,Land use,NaN,Calculated,Input
168,wetlands,-22.280758,Land occupation,Land use,RER,f0efe0f8-35ff-4d74-b084-38b4319c8a84,b2ad6890-c78d-11e6-9d9d-cec0c932ce01,Land use,NaN,Calculated,Input
169,"wetlands, coastal",-22.280758,Land occupation,Land use,RER,4c3b387c-8c28-4848-90fe-39620a81a013,b2ad6890-c78d-11e6-9d9d-cec0c932ce01,Land use,NaN,Calculated,Input


In [17]:
weighted_avg_df.to_csv('data/land_use_rer_cf.csv', index=False)

In [18]:
# Verification
ef_cfs_lu_europe_agri = ef_cfs_lu_lt_europe[ef_cfs_lu_lt_europe['FLOW_name'] == 'from agriculture']
ef_cfs_lu_europe_agri['Weighted CF'] = ef_cfs_lu_europe_agri['CF EF3.1'] * ef_cfs_lu_europe_agri['2022']
rer_cf = ef_cfs_lu_europe_agri['Weighted CF'].sum() / ef_cfs_lu_europe_agri['2022'].sum()
rer_cf == -287.7740232737791

C:\Users\matth\AppData\Local\Temp\ipykernel_44488\126631040.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ef_cfs_lu_europe_agri['Weighted CF'] = ef_cfs_lu_europe_agri['CF EF3.1'] * ef_cfs_lu_europe_agri['2022']


True

# Water use RER CFs

In [30]:
water_use_rer = 36.53 # https://doi.org/10.1007/s11367-017-1333-8 (AWARE, Boulay et al., 2018)

In [31]:
df_water_use = ef_cfs[
    (ef_cfs.LCIAMethod_name == 'Water use')
    & (ef_cfs.LCIAMethod_location.isna())
].copy(deep=True)

In [32]:
df_water_use.LCIAMethod_location = 'RER'

In [33]:
def replace_water_use_cf(new_cf, row):
    if row['CF EF3.1'] < 0:
        return -1.0 * new_cf
    else:
        return new_cf

In [34]:
df_water_use['CF EF3.1'] = df_water_use.apply(lambda row: replace_water_use_cf(water_use_rer, row), axis=1)

In [36]:
df_water_use.to_csv('data/water_use_rer_cf.csv', index=False)